In [2]:
import pandas as pd
import spacy 
import ast

In [9]:
# load the data
student_info = pd.read_csv('../data/00 - mock_student_data_final.csv')
module_info = pd.read_csv('../data/01 - mock_module_info.csv')
department_info = pd.read_csv('../data/02 - mock_department_list.csv')
staff_info = pd.read_csv('../data/03 - mock_staff_info.csv')

In [11]:
def extract_entities(text, label):
    # Extract entities as tuples of (entity text, entity label)
    entities = [(text, label)]
    return entities

def extract_module_entities(module_list):
    module_list = ast.literal_eval(module_list)
    return [(module, 'MODULE') for module in module_list]

In [16]:
# check the first few rows of the student_info dataframe
student_info.head()

,moduleCode,title,description,moduleCredit,department,faculty,gradingBasisDescription,prerequisite,preclusion,attributes,corequisite,lectureHours,tutorialHours,labHours,projectHours,prepHours,semester_01,semester_02,semester_03,semester_04
0,ABM5001,Leadership in Biomedicine,Leadership is fundamental to the success of in...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,0,1,0,0
1,ABM5002,Advanced Biostatistics for Research,This course is served as a concept-based intro...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,0,1,0,0
2,ABM5003,Biomedical Innovation & Enterprise,This course will furnish students with a thoro...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,1,0,0,0
3,ABM5004,Capstone Project,This course encompasses research projects rele...,8.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,1,1,0,0
4,ABM5101,Applied Immunology,Advanced immunological applications play impor...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,1,0,0,0


In [12]:
# extract entities from the student data
student_info['Student_entities'] = student_info['Student_Name'].apply(lambda x: extract_entities(x, 'STUDENT'))
student_info['Faculty_entities'] = student_info['Faculties'].apply(lambda x: extract_entities(x, 'FACULTY'))
student_info['Major_entities'] = student_info['Major'].apply(lambda x: extract_entities(x, 'MAJOR'))
student_info['Module_entities'] = student_info['Modules_Completed'].apply(lambda x: extract_module_entities(x))

In [13]:
#check the first few rows of the student_info dataframe
student_info.head()

,Student_Name,Matric_Number,NRIC,Year,Faculties,Major,Second Major,Modules_Completed,Grades,Student_entities,Faculty_entities,Major_entities,Module_entities
0,Tracy Lewis,A0216920B,XXXXX506Z,1,NUS Business School,Business Administration,NaN,"['ACC1701B', 'DMB1202ACC', 'DMB1201MKT', 'MNO1...","{'ACC1701B': 'B', 'DMB1202ACC': 'A', 'DMB1201M...","[(Tracy Lewis, STUDENT)]","[(NUS Business School, FACULTY)]","[(Business Administration, MAJOR)]","[(ACC1701B, MODULE), (DMB1202ACC, MODULE), (DM..."
1,Andrew Holden,A0225069H,XXXXX799Z,3,YST Conservatory of Music,Music,NaN,"['CFA1111A', 'MUA1190', 'MUA2109', 'MUA1172', ...","{'CFA1111A': 'F', 'MUA1190': 'A', 'MUA2109': '...","[(Andrew Holden, STUDENT)]","[(YST Conservatory of Music, FACULTY)]","[(Music, MAJOR)]","[(CFA1111A, MODULE), (MUA1190, MODULE), (MUA21..."
2,Phillip Bullock,A0228204E,XXXXX194Z,1,College of Design and Engineering,Electrical Engineering,NaN,"['ME1102', 'BN1111', 'PF1103', 'CN1101A', 'ID1...","{'ME1102': 'F', 'BN1111': 'B-', 'PF1103': 'C+'...","[(Phillip Bullock, STUDENT)]","[(College of Design and Engineering, FACULTY)]","[(Electrical Engineering, MAJOR)]","[(ME1102, MODULE), (BN1111, MODULE), (PF1103, ..."
3,Valerie Rivera,A0200778Y,XXXXX150Z,3,Computing,Business Analytics,NaN,"['CS3236R', 'CS1010', 'CP3209', 'CS1010R', 'IS...","{'CS3236R': 'A+', 'CS1010': 'A+', 'CP3209': 'A...","[(Valerie Rivera, STUDENT)]","[(Computing, FACULTY)]","[(Business Analytics, MAJOR)]","[(CS3236R, MODULE), (CS1010, MODULE), (CP3209,..."
4,Robert Hall,A0222508M,XXXXX824Z,3,YST Conservatory of Music,Music,Business Administration,"['MUA1109', 'CFA1101A', 'MUA2255A', 'MUA1172',...","{'MUA1109': 'C+', 'CFA1101A': 'B-', 'MUA2255A'...","[(Robert Hall, STUDENT)]","[(YST Conservatory of Music, FACULTY)]","[(Music, MAJOR)]","[(MUA1109, MODULE), (CFA1101A, MODULE), (MUA22..."


In [18]:
# check first few rows of the module_info dataframe
module_info.head()

,moduleCode,title,description,moduleCredit,department,faculty,gradingBasisDescription,prerequisite,preclusion,attributes,corequisite,lectureHours,tutorialHours,labHours,projectHours,prepHours,semester_01,semester_02,semester_03,semester_04
0,ABM5001,Leadership in Biomedicine,Leadership is fundamental to the success of in...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,0,1,0,0
1,ABM5002,Advanced Biostatistics for Research,This course is served as a concept-based intro...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,0,1,0,0
2,ABM5003,Biomedical Innovation & Enterprise,This course will furnish students with a thoro...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,1,0,0,0
3,ABM5004,Capstone Project,This course encompasses research projects rele...,8.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,1,1,0,0
4,ABM5101,Applied Immunology,Advanced immunological applications play impor...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,NaN,0,0,0,0,0,1,0,0,0


In [23]:
# extract entities from the module data
module_info['Module_entities'] = module_info['moduleCode'].apply(lambda x: extract_entities(x, 'MODULE'))
module_info['Department_entities'] = module_info['department'].apply(lambda x: extract_entities(x, 'DEPARTMENT'))
module_info['Faculty_entities'] = module_info['faculty'].apply(lambda x: extract_entities(x, 'FACULTY'))
# ****

In [24]:
# check first few rows of the module_info dataframe
module_info.head()

,moduleCode,title,description,moduleCredit,department,faculty,gradingBasisDescription,prerequisite,preclusion,attributes,...,labHours,projectHours,prepHours,semester_01,semester_02,semester_03,semester_04,Module_entities,Department_entities,Faculty_entities
0,ABM5001,Leadership in Biomedicine,Leadership is fundamental to the success of in...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0,0,0,0,1,0,0,"[(ABM5001, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]"
1,ABM5002,Advanced Biostatistics for Research,This course is served as a concept-based intro...,2.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0,0,0,0,1,0,0,"[(ABM5002, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]"
2,ABM5003,Biomedical Innovation & Enterprise,This course will furnish students with a thoro...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0,0,0,1,0,0,0,"[(ABM5003, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]"
3,ABM5004,Capstone Project,This course encompasses research projects rele...,8.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0,0,0,1,1,0,0,"[(ABM5004, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]"
4,ABM5101,Applied Immunology,Advanced immunological applications play impor...,4.0,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,Graded,[],[],NaN,...,0,0,0,1,0,0,0,"[(ABM5101, MODULE)]","[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]"


In [25]:
# check first few rows of the department_info dataframe
department_info.head()

,department_id,department,faculty
0,NUSDP0001,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine
1,NUSDP0002,Architecture,College of Design and Engineering
2,NUSDP0003,Accounting,NUS Business School
3,NUSDP0004,Communications and New Media,Arts and Social Science
4,NUSDP0005,History,Arts and Social Science


In [26]:
# extract entities from the department data
department_info['Department_entities'] = department_info['department'].apply(lambda x: extract_entities(x, 'DEPARTMENT'))
department_info['Faculty_entities'] = department_info['faculty'].apply(lambda x: extract_entities(x, 'FACULTY'))


In [28]:
# check first few rows of the department_info dataframe``
department_info.head()

,department_id,department,faculty,Department_entities,Faculty_entities
0,NUSDP0001,NUS Medicine Dean's Office,Yong Loo Lin Sch of Medicine,"[(NUS Medicine Dean's Office, DEPARTMENT)]","[(Yong Loo Lin Sch of Medicine, FACULTY)]"
1,NUSDP0002,Architecture,College of Design and Engineering,"[(Architecture, DEPARTMENT)]","[(College of Design and Engineering, FACULTY)]"
2,NUSDP0003,Accounting,NUS Business School,"[(Accounting, DEPARTMENT)]","[(NUS Business School, FACULTY)]"
3,NUSDP0004,Communications and New Media,Arts and Social Science,"[(Communications and New Media, DEPARTMENT)]","[(Arts and Social Science, FACULTY)]"
4,NUSDP0005,History,Arts and Social Science,"[(History, DEPARTMENT)]","[(Arts and Social Science, FACULTY)]"


In [29]:
# check first few rows of the staff_info dataframe
staff_info.head()

,Employee ID,Employee Name,NRIC,DOB,DOJ,Department_ID,Modules Taught
0,NUSPF12345,Marin Sergio Hernandez,XXXXX479A,1983-02-23T00:00:00Z,2009-10-31T00:00:00Z,Electrical and Computer Engineering,CEG5003
1,NUSPF12346,Kathryn Cordova,XXXXX815A,1985-09-02T00:00:00Z,2009-06-07T00:00:00Z,Civil and Environmental Engineering,ESE2102
2,NUSPF12347,Barbara Sanchez,XXXXX777A,1971-07-30T00:00:00Z,2008-05-09T00:00:00Z,Centre for Language Studies,LAT4201HM
3,NUSPF12348,Bryce Lucas,XXXXX610A,1973-07-20T00:00:00Z,2002-01-17T00:00:00Z,BIZ Dean's Office,DMB1203MNO
4,NUSPF12349,Judith Camacho,XXXXX629A,1991-11-16T00:00:00Z,2000-02-13T00:00:00Z,Economics,EC4401HM


In [31]:
# extract entities from the staff_info data
staff_info['Staff_entities'] = staff_info['Employee Name'].apply(lambda x: extract_entities(x, 'STAFF'))
staff_info['Department_entities'] = staff_info['Department_ID'].apply(lambda x: extract_entities(x, 'DEPARTMENT'))
staff_info['Module_entities'] = staff_info['Modules Taught'].apply(lambda x: extract_entities(x, 'MODULE'))

In [32]:
# check first few rows of the staff_info dataframe
staff_info.head()

,Employee ID,Employee Name,NRIC,DOB,DOJ,Department_ID,Modules Taught,Staff_entities,Department_entities,Module_entities
0,NUSPF12345,Marin Sergio Hernandez,XXXXX479A,1983-02-23T00:00:00Z,2009-10-31T00:00:00Z,Electrical and Computer Engineering,CEG5003,"[(Marin Sergio Hernandez, STAFF)]","[(Electrical and Computer Engineering, DEPARTM...","[(CEG5003, MODULE)]"
1,NUSPF12346,Kathryn Cordova,XXXXX815A,1985-09-02T00:00:00Z,2009-06-07T00:00:00Z,Civil and Environmental Engineering,ESE2102,"[(Kathryn Cordova, STAFF)]","[(Civil and Environmental Engineering, DEPARTM...","[(ESE2102, MODULE)]"
2,NUSPF12347,Barbara Sanchez,XXXXX777A,1971-07-30T00:00:00Z,2008-05-09T00:00:00Z,Centre for Language Studies,LAT4201HM,"[(Barbara Sanchez, STAFF)]","[(Centre for Language Studies, DEPARTMENT)]","[(LAT4201HM, MODULE)]"
3,NUSPF12348,Bryce Lucas,XXXXX610A,1973-07-20T00:00:00Z,2002-01-17T00:00:00Z,BIZ Dean's Office,DMB1203MNO,"[(Bryce Lucas, STAFF)]","[(BIZ Dean's Office, DEPARTMENT)]","[(DMB1203MNO, MODULE)]"
4,NUSPF12349,Judith Camacho,XXXXX629A,1991-11-16T00:00:00Z,2000-02-13T00:00:00Z,Economics,EC4401HM,"[(Judith Camacho, STAFF)]","[(Economics, DEPARTMENT)]","[(EC4401HM, MODULE)]"


In [ ]:
# write the data to new csv files
# ****